<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install pyhealth

In [27]:
root= "https://storage.googleapis.com/pyhealth/eicu-crd/2.0"

In [28]:
from pyhealth.datasets import eICUDataset
dataset = eICUDataset(
        root= root,
        tables=["diagnosis", "medication", "treatment", "physicalExam", "admissionDx"],
        code_mapping = {},
        dev = True
    )


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Parsing patients: 100%|██████████| 3671/3671 [00:10<00:00, 360.17it/s]


finish basic patient information parsing : 11.146253108978271s
finish parsing diagnosis : 133.19076490402222s
finish parsing medication : 178.42491340637207s
finish parsing treatment : 95.58994436264038s
finish parsing physicalExam : 166.43167328834534s
finish parsing admissionDx : 122.16718602180481s


Mapping codes: 100%|██████████| 3671/3671 [00:00<00:00, 10433.35it/s]


In [29]:
dataset.stat()


Statistics of base dataset (dev=True):
	- Dataset: eICUDataset
	- Number of patients: 3671
	- Number of visits: 5000
	- Number of visits per patient: 1.3620
	- Number of events per visit in diagnosis: 7.3732
	- Number of events per visit in medication: 39.7860
	- Number of events per visit in treatment: 0.0000
	- Number of events per visit in physicalExam: 14.3410
	- Number of events per visit in admissionDx: 2.7186



'\nStatistics of base dataset (dev=True):\n\t- Dataset: eICUDataset\n\t- Number of patients: 3671\n\t- Number of visits: 5000\n\t- Number of visits per patient: 1.3620\n\t- Number of events per visit in diagnosis: 7.3732\n\t- Number of events per visit in medication: 39.7860\n\t- Number of events per visit in treatment: 0.0000\n\t- Number of events per visit in physicalExam: 14.3410\n\t- Number of events per visit in admissionDx: 2.7186\n'

In [30]:
dataset.info()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



In [31]:
from pyhealth.tasks import mortality_prediction_eicu_fn
dataset_sample = dataset.set_task(mortality_prediction_eicu_fn)

Generating samples for mortality_prediction_eicu_fn: 100%|██████████| 3671/3671 [00:00<00:00, 69873.34it/s]


In [32]:
dataset_sample.samples[0]

{'visit_id': '146852',
 'patient_id': '002-1043+133311',
 'conditions': [['578.9',
   'K92.2',
   '714.0',
   'M06.9',
   '710.0',
   'M32.10',
   '411.1',
   'I20.0',
   '491.20',
   'J44.9']],
 'procedures': [['notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Score/scored',
   'notes/Progress Notes/Physical Exam/Physical Exam Obtain Options/Performed - Structured',
   'notes/Progress Notes/Physical Exam/Physical Exam/Constitutional/Weight and I&O/Weight (kg)/Admission',
   'notes/Progress Notes/Physical Exam/Physical Exam/Constitutional/Weight and I&O/Weight (kg)/Current',
   'notes/Progress Notes/Physical Exam/Physical Exam/Constitutional/Weight and I&O/Weight (kg)/Delta',
   'notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Motor Score/6',
   'notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Eyes Score/4',
   'notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Verbal Score/5']],
 'drugs': [['2 ML VIAL : ONDANSETRON HCL 4 MG/

In [33]:
dataset_sample.stat()

Statistics of sample dataset:
	- Dataset: eICUDataset
	- Task: mortality_prediction_eicu_fn
	- Number of samples: 320
	- Number of patients: 278
	- Number of visits: 320
	- Number of visits per patient: 1.1511
	- conditions:
		- Number of conditions per sample: 9.5813
		- Number of unique conditions: 467
		- Distribution of conditions (Top-10): [('518.81', 84), ('J96.00', 83), ('038.9', 72), ('584.9', 70), ('N17.9', 70), ('A41.9', 68), ('491.20', 64), ('J44.9', 64), ('458.9', 61), ('I95.9', 61)]
	- procedures:
		- Number of procedures per sample: 19.8344
		- Number of unique procedures: 55
		- Distribution of procedures (Top-10): [('notes/Progress Notes/Physical Exam/Physical Exam Obtain Options/Performed - Structured', 319), ('notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Score/scored', 316), ('notes/Progress Notes/Physical Exam/Physical Exam/Constitutional/Weight and I&O/Weight (kg)/Current', 280), ('notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Mo

"Statistics of sample dataset:\n\t- Dataset: eICUDataset\n\t- Task: mortality_prediction_eicu_fn\n\t- Number of samples: 320\n\t- Number of patients: 278\n\t- Number of visits: 320\n\t- Number of visits per patient: 1.1511\n\t- conditions:\n\t\t- Number of conditions per sample: 9.5813\n\t\t- Number of unique conditions: 467\n\t\t- Distribution of conditions (Top-10): [('518.81', 84), ('J96.00', 83), ('038.9', 72), ('584.9', 70), ('N17.9', 70), ('A41.9', 68), ('491.20', 64), ('J44.9', 64), ('458.9', 61), ('I95.9', 61)]\n\t- procedures:\n\t\t- Number of procedures per sample: 19.8344\n\t\t- Number of unique procedures: 55\n\t\t- Distribution of procedures (Top-10): [('notes/Progress Notes/Physical Exam/Physical Exam Obtain Options/Performed - Structured', 319), ('notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/Score/scored', 316), ('notes/Progress Notes/Physical Exam/Physical Exam/Constitutional/Weight and I&O/Weight (kg)/Current', 280), ('notes/Progress Notes/Physical E

In [34]:
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader

train_ds,valid_ds, test_ds = split_by_patient(dataset_sample, [0.8, 0.1, 0.1])

train_loader = get_dataloader(train_ds, batch_size=32, shuffle=True)
valid_loader = get_dataloader(valid_ds, batch_size=32, shuffle=True)
test_loader = get_dataloader(test_ds, batch_size=32, shuffle=True)

In [35]:
from pyhealth.trainer import Trainer

/usr/local/lib/python3.10/dist-packages/pyhealth/trainer.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [36]:
from pyhealth.models import Transformer
model = Transformer(
    dataset = dataset_sample,
    feature_keys = ['conditions', 'procedures', 'drugs'],
    label_key = 'label',
    mode = 'binary'
)

In [37]:
model

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(469, 128, padding_idx=0)
    (procedures): Embedding(57, 128, padding_idx=0)
    (drugs): Embedding(234, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GEL

In [39]:
trainer = Trainer(model=model)
trainer.train(
    train_dataloader=train_loader,
    test_dataloader = test_loader,
    epochs = 100
)

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(469, 128, padding_idx=0)
    (procedures): Embedding(57, 128, padding_idx=0)
    (drugs): Embedding(234, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GEL

INFO:pyhealth.trainer:Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(469, 128, padding_idx=0)
    (procedures): Embedding(57, 128, padding_idx=0)
    (drugs): Embedding(234, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
       

Metrics: None


INFO:pyhealth.trainer:Metrics: None


Device: cpu


...

INFO:pyhealth.trainer:roc_auc: 0.6111


f1: 0.0000


INFO:pyhealth.trainer:f1: 0.0000


loss: 1.0656


INFO:pyhealth.trainer:loss: 1.0656


In [40]:
from pyhealth.metrics.binary import binary_metrics_fn
y_true, y_prob, loss = trainer.inference(test_loader)
binary_metrics_fn(y_true, y_prob, metrics=["pr_auc", "roc_auc","f1","accuracy", "balanced_accuracy", "recall"])
# Судя по accuracy имеем дисбаланс классов

Evaluation: 100%|██████████| 2/2 [00:00<00:00, 58.83it/s]


{'pr_auc': 0.1486291486291486,
 'roc_auc': 0.611111111111111,
 'f1': 0.0,
 'accuracy': 0.9090909090909091,
 'balanced_accuracy': 0.5,
 'recall': 0.0}

In [41]:
from pyhealth.tasks import readmission_prediction_eicu_fn2
data_sample = dataset.set_task(readmission_prediction_eicu_fn2)
# другие модели почему-то не работали с данным набором данных


Generating samples for readmission_prediction_eicu_fn2: 100%|██████████| 3671/3671 [00:00<00:00, 127955.67it/s]


IndexError: ignored